In [1]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

In [69]:
df = pd.read_csv("02_Test_Data_Set/edu_enrollees.csv")
df.head(2)

,enrollee_id,city,city_development_index,gender,relevant_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,target,Xgrp
0,8949.0,city_103,0.920,Male,Has relevant experience,no_enrollment,Graduate,STEM,>20,NaN,NaN,1,36.0,1.0,train
1,29725.0,city_40,0.776,Male,No relevant experience,no_enrollment,Graduate,STEM,15,50-99,Pvt Ltd,>4,47.0,0.0,train


In [70]:
base = df.drop(["city", "company_size", "company_type"], axis = 1)
base.head(2)

,enrollee_id,city_development_index,gender,relevant_experience,enrolled_university,education_level,major_discipline,experience,last_new_job,training_hours,target,Xgrp
0,8949.0,0.920,Male,Has relevant experience,no_enrollment,Graduate,STEM,>20,1,36.0,1.0,train
1,29725.0,0.776,Male,No relevant experience,no_enrollment,Graduate,STEM,15,>4,47.0,0.0,train


In [71]:
base = base.dropna()
base.isna().sum()

enrollee_id               0
city_development_index    0
gender                    0
relevant_experience       0
enrolled_university       0
education_level           0
major_discipline          0
experience                0
last_new_job              0
training_hours            0
target                    0
Xgrp                      0
dtype: int64

In [72]:
base["experience"].unique()

array(['>20', '15', '13', '7', '5', '16', '4', '11', '<1', '18', '19',
       '12', '10', '9', '2', '6', '14', '3', '8', '20', '17', '1'],
      dtype=object)

In [73]:
# base = base.loc[~base["experience"].isin([">20", "<1"]), ] # isin 사용해서 한줄로 해결
base = base.loc[~(base["experience"] == ">20"), :]
base = base.loc[~(base["experience"] == "<1"), :]
base["experience"] = base["experience"].astype("int")
base["experience"].unique()

array([15, 13,  7,  5, 16,  4, 11, 18, 19, 12, 10,  9,  2,  6, 14,  3,  8,
       20, 17,  1])

In [74]:
base["last_new_job"].unique()

array(['>4', '1', '2', '3', 'never', '4'], dtype=object)

In [75]:
base = base.loc[~(base["last_new_job"] == ">4"), :]
base = base.loc[~(base["last_new_job"] == "never"), :]
base["last_new_job"] = base["last_new_job"].astype("int")
base["last_new_job"].unique()

array([1, 2, 3, 4])

In [76]:
base.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7522 entries, 8 to 19154
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   enrollee_id             7522 non-null   float64
 1   city_development_index  7522 non-null   float64
 2   gender                  7522 non-null   object 
 3   relevant_experience     7522 non-null   object 
 4   enrolled_university     7522 non-null   object 
 5   education_level         7522 non-null   object 
 6   major_discipline        7522 non-null   object 
 7   experience              7522 non-null   int64  
 8   last_new_job            7522 non-null   int64  
 9   training_hours          7522 non-null   float64
 10  target                  7522 non-null   float64
 11  Xgrp                    7522 non-null   object 
dtypes: float64(4), int64(2), object(6)
memory usage: 764.0+ KB


## Q1.

In [77]:
base.head(2)

,enrollee_id,city_development_index,gender,relevant_experience,enrolled_university,education_level,major_discipline,experience,last_new_job,training_hours,target,Xgrp
8,27107.0,0.92,Male,Has relevant experience,no_enrollment,Graduate,STEM,7,1,46.0,1.0,train
11,23853.0,0.92,Male,Has relevant experience,no_enrollment,Graduate,STEM,5,1,108.0,0.0,train


In [79]:
tb_q1 = pd.crosstab(base["relevant_experience"], base["target"])
tb_q1

target,0.0,1.0
relevant_experience,,
Has relevant experience,4790,1319
No relevant experience,872,541


In [28]:
A = tb_q1.iloc[1, 1] / tb_q1.iloc[1, :].sum()
A

0.38287331917905165

In [29]:
B = tb_q1.iloc[0, 1] / tb_q1.iloc[0, :].sum()
B

0.21591095105581928

In [30]:
round(A / B, 2)

1.77

## Q2.

In [31]:
dum = base[["gender", "relevant_experience", "enrolled_university", "education_level", "major_discipline"]]
dum.head(2)

,gender,relevant_experience,enrolled_university,education_level,major_discipline
8,Male,Has relevant experience,no_enrollment,Graduate,STEM
11,Male,Has relevant experience,no_enrollment,Graduate,STEM


In [33]:
dum["gender"].unique()

array(['Male', 'Female', 'Other'], dtype=object)

In [36]:
temp = []
for i in range(5):
    temp = temp + [pd.get_dummies(dum.iloc[:, i])]
    last_col = temp[i].columns[-1]
    temp[i] = temp[i].drop(last_col, axis = 1)
dum = pd.concat(temp, axis=1)
dum.head(2)

,Female,Male,Has relevant experience,Full time course,Part time course,Graduate,Masters,Arts,Business Degree,Humanities,No Major,Other
8,0,1,1,0,0,1,0,0,0,0,0,0
11,0,1,1,0,0,1,0,0,0,0,0,0


In [37]:
temp2 = base[["target", "Xgrp", "city_development_index", "experience", "last_new_job", "training_hours"]]
job2 = pd.concat([temp2, dum], axis=1)
job2.head(2)

,target,Xgrp,city_development_index,experience,last_new_job,training_hours,Female,Male,Has relevant experience,Full time course,Part time course,Graduate,Masters,Arts,Business Degree,Humanities,No Major,Other
8,1.0,train,0.92,7,1,46.0,0,1,1,0,0,1,0,0,0,0,0,0
11,0.0,train,0.92,5,1,108.0,0,1,1,0,0,1,0,0,0,0,0,0


In [45]:
len(job2.iloc[:, 2:].columns)

16

In [47]:
model_log = LogisticRegression(C = 100000, max_iter = 1000, solver="liblinear", random_state=123)
model_log.fit(X = job2.iloc[:, 2:], y = job2["target"])

LogisticRegression(C=100000, max_iter=1000, random_state=123,
                   solver='liblinear')

In [60]:
round(np.exp(model_log.coef_)[0].max(), 2)

1.67

## Q3.

In [61]:
job2_tr = job2.loc[job2["Xgrp"] == "train", :]
job2_t = job2.loc[job2["Xgrp"] == "test", :]

In [62]:
model_kn = KNeighborsClassifier(n_neighbors = 5).fit(X = job2_tr.iloc[:, 2:], y = job2_tr["target"])
pred = model_kn.predict(job2_t.iloc[:, 2:])

In [63]:
from sklearn.metrics import accuracy_score

In [65]:
round(accuracy_score(y_pred = pred, y_true = job2_t["target"]), 2)

0.71